# Forgot User Password

## Initialize the server

In [ ]:
# stdlib

# syft absolute
import syft as sy
from syft import SyftError
from syft import SyftSuccess

server = sy.orchestra.launch(
    name="test-datasite-1",
    dev_mode=True,
    create_producer=True,
    n_consumers=3,
    reset=True,
    port=8081,
)

## Register a new user

In [ ]:
datasite_client = server.login(email="info@openmined.org", password="changethis")
res = datasite_client.register(
    email="new_syft_user@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="New User",
)

if not isinstance(res, SyftSuccess):
    raise Exception(f"Res isn't SyftSuccess,  its {res}")

### Ask for a password reset - Notifier disabled Workflow

### Call for users.forgot_password

In [ ]:
guest_client = server.login_as_guest()
res = guest_client.users.forgot_password(email="new_syft_user@openmined.org")

if not isinstance(res, SyftSuccess):
    raise Exception(f"Res isn't SyftSuccess,  its {res}")

### Admin generates a temp token

In [ ]:
temp_token = datasite_client.users.request_password_reset(
    datasite_client.notifications[-1].linked_obj.resolve.id
)

if not isinstance(temp_token, str):
    raise Exception(f"temp_token isn't a string,  its {temp_token}")

### User use this token to reset password

In [ ]:
res = guest_client.users.reset_password(token=temp_token, new_password="Password123")

if not isinstance(res, SyftSuccess):
    raise Exception(f"Res isn't SyftSuccess,  its {res}")

In [ ]:
new_user_session = server.login(
    email="new_syft_user@openmined.org", password="Password123"
)

if isinstance(new_user_session, SyftError):
    raise Exception(f"Res isn't SyftSuccess,  its {new_user_session}")